<img src="https://javier.rodriguez.org.mx/itesm/2014/tecnologico-de-monterrey-blue.png" alt="Logo" style="width:500px;"/>
</br>
</br>
<center> 
    <b style="font-size:24pt"> Algorithm Design and Analysis </b>
</center>
</br>
<center> 
    <b style="font-size:14pt"> The 8 Queens Problem </b>
</center>

</br>
</br>

<center> 
    TC1031 - Programming of Data Structures and Fundamental Algorithms
</center>

<center> 
    Gruop 504
</center>

</br>
</br>
</br>
</br>

<center> 
    Joaquín Badillo Granillo
</center>
<center> 
    A01026364
</center>

</br>
</br>
</br>
</br>
$\newcommand{\bigO}[1]{\mathcal{O}\left(#1\right)}$

<center> 
    <b> Under the instruction of </b>
</center>
<center> 
    Esteban Castillo Juárez
</center>

# The 8 Queens Problem

Given an $8 \times 8$ chess board, we want to find all the possible ways to fit 8 queens in it without them threatening each other (see the valid solution below). To solve this problem we are required to use 2 distinct algorithm design techniques: *Brute Force* and either *Divide & Conquer* or *Dynamic Programming*.

**Valid solution example:**

|    |  0 |  1 |  2 |  3 |  4 |  5 |  6 |  7 |
|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
|  0 | .  | .  | .  | .  | .  | ♛  | .  |  . |
|  1 | .  | .  | .  | ♛  | .  | .  | .  |  . |
|  2 | .  | .  | .  | .  | .  | .  | ♛  |  . |
|  3 | ♛  | .  | .  | .  | .  | .  | .  |  . |
|  4 | .  | .  | .  | .  | .  | .  | .  |  ♛ |
|  5 | .  | ♛  | .  | .  | .  | .  | .  |  . |
|  6 | .  | .  | .  | .  | ♛  | .  | .  |  . |
|  7 | .  | .  | ♛  | .  | .  | .  | .  |  . |

To avoid any possible error when trying to print unicode characters, the programs will denote queens `♛` by `Q` and the `.` character is regarded as a blank space.

# Model of Computation

Since it is of our interest to evaluate the efficiency of the algorithms, we need a way to compare the time they might need to execute in a way that is independent of hardware. But a machine-independent approach requires some assumptions, as different architecutres may support some operations faster than others. Thus, to be as precise as possible, throughout this notebook we are going to consider a Random Access Machine Model which we will develop so that it is as specific as possible on the number of steps done by a processor. Moreover, the usual abstraction behind libraries allows us to consider a general time complexity for the functions and objects it supports but it is rather difficult to determine a specific complexity, thus for objects like unordered_sets we will just consider single operation insertions.

## The Random Access Machine

We will break down the possible instructions an algorithm in this notebook might require so that it is clear for the reader the number of steps required for a program in the abstract model of computation we considered adequated.

### Mathematical and Logical Manipulation

* Basic arithmetic operations (`+`, `-`, `*`, `/`) take 1 instruction to execute
    * Increment `++` and decrement `--` operations take 1 instruction to execute (including iterators).
    * Multiplication and division assignment (`*=`, `/=`) take 1 instruction to execute.
* Applying the modulo operator `a % b` takes 1 instruction to execute.
* Comparison by value (`<`, `<=`, `==`, `!=`, `>=` `>`) is done in a single instruction
    * Such a comparison applies to constant strings as well.
* Generating random numbers requires an instruction
    * Setting a random seed is also a single instruction operation
* Logical operators (`&&`, `||`, `!`) require 1 instruction.
* If, else if and else statements do not require instructions
    * When finding one of these statements it is required to count the operations used to get a logical value.

### Computer Memory and Storage

* Accessing memory takes 1 instruction to execute
    * This includes using the `->` operator.
* Declaring a variable (without initializaion) takes 1 instruction.
* When assigning a value to a primitive data type an instruction is required.
    * Constructors are not single instruction operations for arrays and hash sets, for these data types the number of elements used to construct them corresponds to the number of instructions done.
* Accessing the members of a struct or an object (attributes and methods) can be done in 1 instruction
    * Both the `.` and `->` operators require a single instruction in this scenario.
* Dereferencing a pointer or iterator (`*x`) takes a single instruction.
* Advancing an iterator by a fixed amount is a single step operation
    * std::advance(it, x) takes a single instruction.

#### Unordered Sets

* An iterator to the first element of an unordered set is obtainable in a single instruction
    * That is `X.begin()` or `X -> begin()` take 2 instructions, one to access the method and another to obtain the iterator.
* The end of an unordered set is obtainable in a single instruction
    * `X.end()` or `X -> end()` take 2 instructions to execute.
* The size of an unordered set is accesible in a single instruction
    * `X.size()` and `X -> size()` are both completed in 2 instructions.
* Inserting on an unordered set is a single instruction operation
    * `X.insert(x)` and `X -> insert(x)` require 2 instructions.
* Deleting from an unordered set, using either a value or iterator takes a single instruction
    * `X.erase(x)` and `X -> erase(x)` take 2 instructions.
* Finding a value on an unordered set takes a single instruction
    * `X.find(x)` and `X -> find(x)` are both 2 instruction operations.


#### Time

* Accessing the time (`time(NULL)`) is considered a single step operation.

### I/O

* Printing formatted and constant strings takes a single instruction.
* Opening and closing a file takes a single instruction
    * It takes a single instruction even if the file does not exist yet.
    * Only the open() method takes 1 instruction, accesing it with `.` requires another instruction.
* Each time a string is written to a file 1 instruction is required.
* Each time a string is read from a file 1 instruction is required.

### Miscellaneous

* Including libraries requires no additional instructions.
* Calling functions only takes as many operations as the function required
    * the call itself does not count as an instruction.
    * any process done when passing data by reference or value is not considered in the model.
* Returning a value from a function requires no additional instructions.
* The instructions done by a loop are equal to the operations done inside them
    * `for` loops require an initialization instruction (usually a value is assigned to a variable).

## Considerations

Obviously such an abstraction does not match any real machine, we can start by noticing that most computers add faster than they divide, the increment operation could easily make the addition and then assign the value (2 instructions in the model) but it is considered a single step instruction (due to its constant general complexity), calling functions requires cloning variables and adding the function to the call stack (that cannot possibly be done without any instruction). However, with this model we are trying to establish a convention that can map somewhat directly to our experience using computers, arithmetic is fast, printing relatively small texts in a terminal environment or in a file is super fast as well, to name a few of our considerations.

# Brute Force Solution

For the Brute Force solution we only need to find one valid configuration of the queens, thus we can use a randomized algorithm that chooses arbitrary and unique positions for the queens on the board  until it finds a valid solution. Since we know the 8 queens problem has at least a valid solution, this algorithm should eventually converge to one of them (there are 92 possible solutions).

## Libraries

In [1]:
#include <stdio.h>
#include <stdlib.h>
#include <time.h>

#include <unordered_set>

## Helper Functions

### Get Row

In [2]:
int row(int pos) {
    return pos / 8;
}

#### Specific Time Complexity

Using the previously described model, this function only requires a division, which means it's specific time complexity is
$$\bigO{1}.$$

#### General Time Complexity

Using properties of the asymptotic notation, since this operation always requires a single step (it is a consant time operation) its general time complexity is
$$\bigO{1}.$$

### Get Column

In [3]:
int column(int pos) {
    return pos % 8;
}

#### Specific Time Complexity

The `column` function only returns the result of its input modulo 8, this means that the specific time complexity is
$$\bigO{1}.$$

#### General Time Complexity

With the same argument as before, the general time complexity must be
$$\bigO{1}.$$

### Check Solution

In [4]:
bool checkSolution(std::unordered_set<int> queens) {
    std::unordered_set<int> rows;
    std::unordered_set<int> columns;
    std::unordered_set<int> diagonalsR;
    std::unordered_set<int> diagonalsL;
    
    for (auto it = queens.begin(); it != queens.end(); std::advance(it, 1)) {
        int y = row(*it);
        int x = column(*it);
        rows.insert(y);
        columns.insert(x);
        diagonalsR.insert(y - x);
        diagonalsL.insert(y + x);
    }
    
    return (rows.size() + 
            columns.size() +
            diagonalsR.size() + 
            diagonalsL.size() == 32);
}

### Specific Time Complexity

Let us start by noticing we declare 4 empty sets (we do not initialize them) which requires a total of 4 instructions. Then we start a `for` loop, during its initialization we declare an iterator `it` and assign it the beginning of the given set, this takes 4 instructions counting declaration, access, the `begin()` method and assignation. Then for every iteration of the loop the program has to

* Compare `it != queens.end()` (3 instructions)
* Increment the iterator (1 instruction)
* Create 2 variables, dereference the iterator, get the column or row and asign the result (8 instructions)
* Insert the row and column in the adequate hash set (4 instructions)
* Insert the left and right diagonals, which require an additional arithmetic operation (6 instructions)

The loop iterates over the queen set, thus by denoting the size of the set as $s$, the loop (including its initialization and exlcuding the final comparison as established previously) requires
$$22s + 4$$
basic instructions. Afterwards we will see that $s$ is always 8 when calling this function in the main program since we only check the set after placing 8 random queens.

Finally the function obtains the size of the rows, columns and diagonals sets (8 instructions), adds them up (3 instructions) and compares the result to 32 (1 instruction). Adding up all instructions the specific time complexity of the algorithm as a function of the size $s$ of the set `queens` is
$$\bigO{22s + 20}$$

### General Time Complexity

As we showed that the number of steps increases linearly with the size $s$ of the input with the function $22s+20$, the general time complexity of the algorithm is
$$\bigO{s}$$

### Print Solution

In [5]:
void printSolution(std::unordered_set<int> queens) {
    int position = 0;
    for (unsigned short int i = 0; i < 8; ++i) {
        for (unsigned short int j = 0; j < 8; ++j) {
            if (queens.find(position) != queens.end())
                printf("Q ");
            else
                printf(". ");
            ++position;
        }
        printf("\n");
    }
}

### Specific Time Complexity

This function starts by intitializing a variable to 0 (`int position = 0`) which requires 2 instructions, then in a `for` loop it initializes another variable `i` to 0 and in each iteration of the loop the algorithm:

* compares `i < 8` (1 instruction)
* increases the variable `i` (1 instruction)
* starts a `for` loop, that initializes a variable `j` to 0 (2 instructions)
    * in each iteration of the nested loop:
    * we search for `position` in the set (2 instructions)
    * we obtain the end of the set (2 instructions)
    * we compare the result of the search with the end (1 instruction)
    * a character and a space is printed depending on the result of the expression (1 instruction)
    * `position` is incremented (1 instruction)
* a line break is printed (1 instruction)

The nested loop including its initialization takes 
$$2 + 7 \times 8 = 58$$
instructions, since it takes 8 iterations. Therefore the outer `for` loop does $5 + 58 = 63$ insstructions per cycle, since it taes 2 instructions to initialize it and it loops for 8 iterations, the total number of instructions required by the loops is
$$2 + 63 \times 8 = 506.$$
After adding the first 2 instructions we deduce that the specific time complexity is independent of the size of the input and its exactly
$$\bigO{508}.$$

### General Time Complexity

Given the fact that the number of steps required by the algorithm is independent of the size of the input, it always takes a constant number of steps and in big-O notation it is expressed as
$$\bigO{1}.$$

## Main

In [6]:
srand(time(NULL));

bool solved = false;
int iterations = 0;

// Board used to get random positions.
std::unordered_set<int> positions({
    0,  1,  2,  3,  4,  5,  6,  7,
    8,  9,  10, 11, 12, 13, 14, 15, 
    16, 17, 18, 19, 20, 21, 22, 23, 
    24, 25, 26, 27, 28, 29, 30, 31, 
    32, 33, 34, 35, 35, 36, 37, 38,
    39, 40, 41, 42, 43, 44, 45, 46, 
    47, 48, 49, 50, 51, 52, 53, 54, 
    56, 57, 58, 59, 60, 61, 62, 63
});

while (!solved) {
    ++iterations;
    std::unordered_set<int> queens;
    
    for (int i = 0; i < 8; ++i) {
        std::unordered_set<int>::iterator it = positions.begin();
        std::advance(it, rand() % positions.size());
        queens.insert(*it);
        positions.erase(it);    
    }
    
    solved = checkSolution(queens);
    if (solved) {
        printf("Valid solution found in %i iteration(s)\n", iterations);
        printSolution(queens);
    }
    else {
        // Restore set
        for (auto it = queens.begin(); it != queens.end(); std::advance(it, 1))
            positions.insert(*it);
    }
}

Valid solution found in 7726806 iteration(s)
. . Q . . . . . 
. . . . . . Q . 
. Q . . . . . . 
. . . . . . . Q 
. . . . Q . . . 
Q . . . . . . . 
. . . Q . . . . 
. . . . . Q . . 


### Worst Case

Let $\mathcal{Q}$ be a sequence of $n$ elements
$$ \mathcal{Q} = \langle q_{1}, q_{2}, q_{3}, \dots, q_{n} \rangle, $$
where each element
$$
    q_{i} = \left\{
    \begin{pmatrix} x_{i_1}\\ y_{i_1} \end{pmatrix}, 
    \begin{pmatrix} x_{i_2}\\ y_{i_2} \end{pmatrix},
    \dots,
    \begin{pmatrix} x_{i_8}\\ y_{i_8} \end{pmatrix}
    \right\}
$$
is a set of ordered pairs that has cardinality 8. The ordered pairs represent coordinates and in this case we may define them as natural numbers in the range $[0, 7]$.

Using the `column()` and `row()` functions in a mathematical manner, there is a mapping from the positions obtained arbitrarily by the program and any $q_{i}$. Therefore the sequence $\mathcal{Q}$ can represent all the arrangements of queens obtained by this algorithm and $q_{n}$ has to be a mapping from the valid solution found before its termination. Such a definition allows us to evaluate the complexity of the program in terms of the number of arrangements used ($n$) to find a valid solution. We will focus on the worst case, when $n$ is really large but its not determined as a specific value and since the program does not save previously used configurations (as that would use way too much memory) there might exist an element in the sequence $q_{j}$, such that $q_{i} = q_{j}$, that is repetition in the sequence $\mathcal{Q}$ is possible and the worst case is not limited to the 
$$C^{64}_{8} - 91 \approx4\times10^9$$
invalid configurations of the queens (plus the iteration that yields a solution). Thus the worst case is that $n$ grows towards $\infty$.

#### Specific Time Complexity

The main function obtains the time (1 instruction) and sets a random seed using the obtained value (1 instruction). Following this, a boolean variable `solved` is declared and initialized to false (2 instructions), then an integer variable `iterations` is declared and initialized to 0 (2 instructions). A set holding all the possible positions in a chess board is declared an initizialed, since it contains 64 positions this requires 65 instructions (the declaration and constructor). Afterwards, in a `while` loop, each iteration:

* negates `solved` (1 instruction)
* initializes a set (1 instruction)
* in a `for` loop a variable `i` is declared and initialized to 0 (2 instructions)
    * in each one of the 8 iterations of the `for` loop:
    * a comparison `i < 8` is evaluated (1 instruction)
    * `i` is incremented (1 instruction)
    * an iterator is declared and initialized with `positions.begin()` (4 instructions)
    * a random value is obtained and its modulo with the size of `positions` is obtained (4 instructions)
    * the iterator advances by the random amount (1 instruction)
    * the value of the iterator is inserted in the set of queens (3 instructions)
    * the iterator is used to delete an element from the set of positions (2 instructions)
* after the `for` loop ends a set of 8 queens is checked ($22\times 8 + 20 = 196$ instructions)
* the state of the solution is assigned to solved (1 instruction)
* in the worst case we did not solve the board so we restore the set ($4 + 7\times8 = 60$ instructions)
* if the board was solved we have to print a formatted text and the solution (509 instructions)

The inner `for` loop requires 16 instructions per cycle, since it is done 8 times the number of instructions needed by this loop (including the initialization step) is
$$2 + 16\times8 = 130.$$
With this information, we can deduce that the outer `while` loop requires 391 instructions in $n-1$ iterations, where $n$ is the size of $\mathcal{Q}$ as mentioned before. Once it finds a solution it requires 839 instructions in one iteration and thus the whole `while` loop requires
$$391\left(n-1\right)+839 = 391n + 448$$
instructions. Hence the specific time complexity of the program is
$$\bigO{391n + 519}.$$

#### General Time Complexity

Since the number of instructions increases linearly with the number of iterations required to find a valid random solution, the general time complexity as a function of $n$ is
$$\bigO{n}.$$

# Dynamic Programming Solution

It is obvious that for this problem 2 queens cannot be placed in the same row or column, thus a simple divide and conquer approach could be to place queens on each column and memoize the row, so that we reduce the possibilities dramatically: 

* First we can put a queen on any of the 8 rows of the first column 

* Then we only have 7 valid rows for the second column

* Afterwards there are 6 rows for the third... 

* In general, for the $i$th column there are $8 - i$ possible positions to choose
    
Using the multiplication principle, this means the dynamic programming approach described has as little as $8!\approx 40 000$ possibilities to check. We can even save some time by checking if any piece is threatened through the diagonals, given the fact that this can be computed in constant time and we could use recursion to backtrack. This is nothing for today's computing power and if you had the patience to wait for a solution with the Brute Force approach, this solution might astonish you (or it might not surprise you since the order of magnitude on the possibilites decreases at least by 5 orders of magnitude)

The idea behind this approach and the reason it may be considered a Dynamic Programming solution is that it solves a simpler subproblem: placing $i$ queens on an $8 \times i$ board; which is then used to try and solve the next subproblem: placing $i+1$ queens on an  $8 \times \left(i + 1\right)$ board. Since not all solutions to an $8 \times i$ allow for valid solutions of the next subproblem we include the backtracking, while memoizing the valid rows and diagonals in sets to avoid doing unecessary computations: once a queen threatens a row or diagonal, no other queen can be placed on that row and diagonal. Since we are placing a queen per column we are not concerned by storing the valid columns. 

To show the results it is way more efficient to store them in a file (`solutions.txt.`) than printing them, since we can avoid doing the work again and check them without executing the program. Now, to avoid the program from writing the file again we can just check if the list of solutions in the file is complete, this could be done traversing the file but that operation is relatively expensive as it grows with the size of the file; a much more efficient approach (in terms of time complexity) is to sacrifice some disk space and a constant number of steps writing to another file (`count.txt`) which will hold the number of solutions found; given the fact that there are only 92 solutions to the 8 Queens Problem we just need to check if that is the value stored in the file, otherwise we make the computations.

A possible improvement we won't delve into is using multiple threads to obtain results faster with concurrency. The reason we did not take this approach is the fact that there is a critical region: all threads want to append their solutions to the same file. We could avoid this problem creating a temporal file per thread and after the main execution call the `cat` command to merge the results, then `rm` the temporal files, however a Windows system will break if we try to execute bash commands. Another improvement may be to consider a way to resume the execution given we store the number of solutions found and we can start at that solution, however making an association between natural numbers and the configuration of the queens is not that straight forward; even though one could read the file and determine the last configuration found, finding a way to resume the execution is not that straight forward as a stack of configurations is stored due to the recursion in the execution, furthermore the program might have halted previously while writing the solution to the file and thus it may be incomplete in the text file.

## Aditional Libraries

In [7]:
// Add file manipulation libraries
#include <fstream>
#include <string>

## Helper Functions

### Append Solution

In [8]:
void appendSolution(std::unordered_set<int> queens) {
    std::ofstream out;
    out.open("solutions.txt", std::ios_base::app);
    
    int position = 0;
    for (unsigned short int i = 0; i < 8; ++i) {
        for (unsigned short int j = 0; j < 8; ++j) {
            if (queens.find(position) != queens.end())
                out << "Q ";
            else
                out << ". ";
            ++position;
        }
        out << "\n";
    }
    out.close();
}

#### Specific Time Complexity

Notice that there is a correspondence between appending the solution to the file and the previous function used to print a solution to the terminal. Since our model treats printing in a terminal as an equivalent operation to writing on a file (a single instruction is required for constant strings), this function requires the same steps with the additional operations of declaring the output file, opening it in append mode and closing it, which are just 5 more instructions. Thus the specific time complexity is
$$\bigO{513}.$$

#### General Time Complexity

Similarly the amount of steps required remains constant and therefore the general time complexity is
$$\bigO{1}.$$

## Recursive Solver and Main Function

### Solver

In [9]:
bool solver(std::unordered_set<int> queens,
            int column,
            int row,
            std::unordered_set<int> validRows, 
            std::unordered_set<int> validRightDiagonals, 
            std::unordered_set<int> validLeftDiagonals,
            int* counter) {
    // Remove currently used row
    validRows.erase(row); 

    // Break branch if it tried to use an invalid diagonal
    bool invalidRightDiagonal = validRightDiagonals.find(column - row) == validRightDiagonals.end();
    bool invalidLeftDiagonal = validLeftDiagonals.find(column + row) == validLeftDiagonals.end();
    if (invalidRightDiagonal || invalidLeftDiagonal)
        return false;

    // Remove threatened diagonals
    validRightDiagonals.erase(column - row);
    validLeftDiagonals.erase(column + row);

    // Solution was valid for the column, we can increment it and check if we traversed the board                                    
    if (++column == 8) {
        ++counter[0];
        appendSolution(queens);
        return true;
    }

    // If we haven't traversed, branch for each valid row and try to solve the next column recursively
    for (auto rowIt = validRows.begin(); rowIt != validRows.end(); std::advance(rowIt, 1)) {
        queens.insert((*rowIt) * 8 + column);
        solver(queens, column, *rowIt, validRows, validRightDiagonals, validLeftDiagonals, counter);
        queens.erase((*rowIt) * 8 + column); 
    }
    
    return false;
}

#### Complexity Analysis

To obtain the time complexity of this function we are going to assign a weight to each line of code which corresponds to the number of instructions that are required to execute it. Then with that information, we can create a function that mimics the behaviour of this one, but that recieves an additional integer by reference that serves as an instruction counter. This method is valid, since the size of the board and the number of queens to place is fixed at 8, thus even if we check a lot of possible configurations the execution of the program will always be the same. Even though the size of the queens set and of the valid rows and diagonals varies with each call to the function, the queens array starts at a value of 1 the first time we call the function in main and it increases up to a maximum of 8, for the rows and diagonals, these start with a size of 7 and 14 respectively but they only decrease. Since insertion and deletion was done with sets to get constant time operations the program must be of constant general complexity ($\bigO{1}$). 

Solving a more general $N$ queens problem would not allow for a constant time algorithm and searching through the possible configurations even with this optimized version of the algorithm takes non-polynomial instructions in the number of queens.

#### Weighting


* `validRows.erase(row);`
    * Takes 2 instructions, accessing the `erase()` method and executing it.

* `bool invalidRightDiagonal = validRightDiagonals.find(column - row) == validRightDiagonals.end();`

* `bool invalidLeftDiagonal = validLeftDiagonals.find(column + row) == validLeftDiagonals.end();`

    * take 8 instructions each, declaring a variable, accessing and executing the `find()` method, doing the arithmetic operation, comparing, accesing and executing the `end()` method, and finally assigning the result.
    
* `if (invalidRightDiagonal || invalidLeftDiagonal) {return false;}`

    * takes 1 instruction always, the logical operation. Returning a value and the if statement itself are not counted.
    
* `validRightDiagonals.erase(column - row);`

* `validLeftDiagonals.erase(column + row);`

    * take 3 instructions each, accesing the method, the arithmetic operation and executing the deletion.
    
*  `if (++column == 8) {`
    
    * takes 2 instructions, increment and comparison.

* `++counter[0];`
    
    * takes 2 instructions, accesing memory and incrementing the value.
    
* `appendSolution(queens);`
    
    * takes 513 instructions as showed previously.
  
* `return true; }
    
    * returning constant values does not count in this model.

* `for (auto rowIt = validRows.begin(); rowIt != validRows.end(); std::advance(rowIt, 1)){`
    
    * initialization requires 4 instructions
    
    * comparison requires 3 instructions
    
    * increment requires 1 instruction

* `queens.insert((*rowIt) * 8 + column);`

    * takes 5 instructions

* `solver(queens, column, *rowIt, validRows, validRightDiagonals, validLeftDiagonals, counter);`

    * calling the function does not count as an instruction, the steps required by it will be counted by the program

* `queens.erase((*rowIt) * 8 + column);}`

    * requires 5 instructions
    
* `return false;`
    
    * requires no additional instructions

Therefore the following program will skip the appending of the queens and will increment the number of instructions done (passing it by reference) when calling it.

In [10]:
bool solverCounter(std::unordered_set<int> queens,
            int column,
            int row,
            std::unordered_set<int> validRows, 
            std::unordered_set<int> validRightDiagonals, 
            std::unordered_set<int> validLeftDiagonals,
            int* counter,
            int* instructions) {
    // Remove currently used row
    *instructions += 2; // Weight
    validRows.erase(row); 

    // Break branch if it tried to use an invalid diagonal
    *instructions += 16;// Weight
    bool invalidRightDiagonal = validRightDiagonals.find(column - row) == validRightDiagonals.end();
    bool invalidLeftDiagonal = validLeftDiagonals.find(column + row) == validLeftDiagonals.end();
    
    ++(*instructions); // Weight
    if (invalidRightDiagonal || invalidLeftDiagonal)
        return false;

    // Remove threatened diagonals
    *instructions += 6; // Weight
    validRightDiagonals.erase(column - row);
    validLeftDiagonals.erase(column + row);

    // Solution was valid for the column, we can increment it and check if we traversed the board                                    
    *instructions += 2; // Weight
    if (++column == 8) {
        *instructions += 2; // Weight
        ++counter[0];
        *instructions += 513; // Weight
        //appendSolution(queens);
        return true;
    }

    // If we haven't traversed, branch for each valid row and try to solve the next column recursively
    *instructions += 4; // Weight
    for (auto rowIt = validRows.begin(); rowIt != validRows.end(); std::advance(rowIt, 1)) {
        *instructions += 3; // Loop Comparison Weight
        
        *instructions += 5; // Weight
        queens.insert((*rowIt) * 8 + column);
        
        solverCounter(queens, column, *rowIt, validRows, validRightDiagonals, validLeftDiagonals, counter, instructions);
        
        *instructions += 5; // Weight
        queens.erase((*rowIt) * 8 + column);
        
        *instructions += 1; // Loop Increment Weight
    }
    
    return false;
}

### Main

In [11]:
std::ifstream in("count.txt");
std::string text;
std::getline(in, text);
in.close();

if (text != "92") {
    // Reset file for safety
    std::ofstream solutions("solutions.txt");
    solutions.close();
    
    int counter = 0;
    
    // Try an initial guess for each row of column 0
    for (int row = 0; row < 8; ++row) {
        std::unordered_set<int> queens;
        std::unordered_set<int> validRows({0, 1, 2, 3, 4, 5, 6, 7});
        std::unordered_set<int> validRightDiagonals({-7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7});
        std::unordered_set<int> validLeftDiagonals({0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14});
        queens.insert(row);
        solver(queens, 0, row , validRows, validRightDiagonals, validLeftDiagonals, &counter);
    }
    
    printf("Found %i solutions!\n", counter);
    
    // Write number of solutions to another file
    std::ofstream out("count.txt");
    out << counter;
    out.close();
}
else
    printf("Problem already solved, check the solutions.txt file\n");

Found 92 solutions!


#### Specific Time Complexity

To obtain the specific time complexity we are going to use the same method, add a counter for instructions and increment it with the number of steps done in our model as the program progresses. Yet again we will avoid writing to the file so we will only execute the function when the condition `text != "92"` holds and avoid modifying the `count.txt` file.

* `std::ifstream in("count.txt");`

    * 2 instructions: declaring an input file and opening it.
    
* `std::string text;`

    * 1 instruction: declaring a string
    
* `std::getline(in, text);`

    * 1 instruction: reading a line that holds up to 2 characters ("92") from a file
    
* `in.close();`
    
    * 2 instructions: access method and execute close().

* `if (text != "92") {`

    * 1 instruction: comparison

* `std::ofstream solutions("solutions.txt");`
  
    * 2 instructions: declaring output file and opening it
 
* `solutions.close();`

    * 2 instructions: access and execute `close()` method

* `int counter = 0;`

    * 2 instructions: declare and initialize integer variable
    
* `for (int row = 0; row < 8; ++row) {`
    
    * Initialization requires 2 instructions: declaration and assignment
    
    * Comparison requires 1 instruction
    
    * Increment requires 1 instruction

* `std::unordered_set<int> queens;`

    * 1 instruction: declaration

* `std::unordered_set<int> validRows({0, 1, 2, 3, 4, 5, 6, 7});`

    * 9 instructions: declaration and constructor

* `std::unordered_set<int> validRightDiagonals({-7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7});`
* `std::unordered_set<int> validLeftDiagonals({0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14});`

    * 16 instructions each: declaration and constructor

* `queens.insert(row);`

    * 2 instructions: access and execute insertion method
    
* `solver(queens, 0, row , validRows, validRightDiagonals, validLeftDiagonals, &counter);`
    
    * Calls themselves do not count, but rather the instructions executed by the function
    
    * This function will be replaced by solverCounter to count the steps and a instruction counter will be passed by reference
    
* `printf("Found %i solutions!\n", counter);`

    * 1 instruction: print formatted string

* `std::ofstream out("count.txt");`
   
    * 2 instructions, declare and open
    
* `out << counter;`
    
    * 1 instruction, write constant.

* `out.close()` 
    
    * 2 instructions, access and execute `close()` method
 
Since this algorithm does not support a value given by a user like another number of queens to use it will always be executed equally and thus it cannot grow in size. That means its complexity is constant and therefore the number of steps can be counted by the following program

In [12]:
int instructions = 6; // Weight of commented code
/*std::ifstream in("count.txt");
std::string text;
std::getline(in, text);
in.close();*/

instructions += 1; // Weight
//if (text != "92") {
    instructions += 2; // Weight

    // Reset file for safety
    std::ofstream solutions("solutions.txt");
    
    instructions += 2; // Weight
    solutions.close();
    
    instructions += 2; // Weight
    int counter = 0;
    
    instructions += 2; // Weight
    // Try an initial guess for each row of column 0
    for (int row = 0; row < 8; ++row) {
        instructions += 1; // Comparison Weight
        
        instructions += 1; // Weight
        std::unordered_set<int> queens;
        
        instructions += 9; // Weight
        std::unordered_set<int> validRows({0, 1, 2, 3, 4, 5, 6, 7});
        
        instructions += 16; // Weight
        std::unordered_set<int> validRightDiagonals({-7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7});
        
        instructions += 16; // Weight
        std::unordered_set<int> validLeftDiagonals({0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14});
        
        instructions += 2; // Weight
        queens.insert(row);
        
        solverCounter(queens, 0, row , validRows, validRightDiagonals, validLeftDiagonals, &counter, &instructions);
    }
    
    instructions += 1; // Weight
    printf("Found %i solutions!\n", counter);
    
    instructions += 5; // Weight of file manipulation
    // Write number of solutions to another file
    /*std::ofstream out("count.txt");
    out << counter;
    out.close();*/
//}
//else
    //printf("Problem already solved, check the solutions.txt file\n");
printf("The algorithm will always execute %i instructions", instructions);

Found 92 solutions!
The algorithm will always execute 253717 instructions

As shown by the output and following from the previous argumentation, the specific complexity of the algorithm is
$$\bigO{253717}.$$

#### General Time Complexity

The general time complexity is constant since the input cannot change and therefore it always takes the same steps when the file hasn't been generated, in asymptotic notation the complexity is
$$\bigO{1}.$$

# Conclusion

We were able to design and implement a Brute Force solution to the 8 Queens Problem and optimize it using Dynamic Programming and Backtracking to check only a subset of the possible arrangements of the queens. Since the number of queens and the chess board is not able to change in this implementation, the second solution will always execute the same amount of steps and is therefore a constant time algorithm.

However, it is much more interesting to generalize the problem to $N$ queens on a $N \times N$ board. In this case even though Dyanmic Programming usually allows for non-polynomial algorithms to be optimized to polynomial time, we are unlucky here, the optimized solution without the backtracking is guaranteed to check $N!$ configurations and determining how many branches of the recursion will be cut is a difficult task to get done.